# پروژه : ایجاد یک مدل ماشین لرنینگ و تشخیص احساس متن توسط یک رابط وب

وارد کردن کتاب خانه های مورد نیاز

In [8]:
import re 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

مقدار دهی رابط وب با چند کد ساده و ایجاد بخشی برای وارد کردن متن مورد نظر

In [9]:
import streamlit as st
st.write("# Text Emotions Prediction")
t1 = st.text_input("Enter any text>>: ")

این تابع جهت خواندن دیتاست برای اموزش مدل پروژه است 
در این پروژه دیتا ست از قبل پیش پردازش شده و اماده است و فایل مورد نظر در ابتدا لیبل گذاری شده و سپس متن هر لیبل نوشته شده است
که در اینجا لیبل های اول هر سطر جدا شده و بعنوان تارگت مورد استفاده قرار خواهند گرفت

In [10]:
def read_data(file):
    data = []
    with open(file, 'r')as f:
        for line in f:
            line = line.strip()
            label = ' '.join(line[1:line.find("]")].strip().split())
            text = line[line.find("]")+1:].strip()
            data.append([label, text])
    return data

در این بخش مسیر دیتا ست به تابع داده میشود

In [11]:
file = 'text.txt'
data = read_data(file)
print("Number of instances: {}".format(len(data)))

Number of instances: 7480


این تابع دو ورودی میگیرد اولی برای متن و دومی یک عدد صحیح و یا یک بازه عددی 
کار این تابع جدا سازی حروف کلمات ب اندازه عدد صحیح میباشد و در اخر یک لیست ازحروف یک یا دو یا چند حرفی برمیگرداند 

In [12]:
def ngram(token, n): 
    output = []
    for i in range(n-1, len(token)): 
        ngram = ' '.join(token[i-n+1:i+1])
        output.append(ngram) 
    return output

تابع استخراج ویژگی ها دو ورودی مانند تابع بالا میگیرد عملکرد این تابع ب این صورت است که متن را بر اساس تکرار هایش در ان خط متن ب صورت دیکشنری دراورده و مقداد تکرارش را ب عنوان مقدار اون کلمه قرار میدهد

In [13]:
def create_feature(text, nrange=(1, 1)):
    text_features = [] 
    text = text.lower() 
    text_alphanum = re.sub('[^a-z0-9#]', ' ', text)
    for n in range(nrange[0], nrange[1]+1): 
        text_features += ngram(text_alphanum.split(), n)    
    text_punc = re.sub('[a-z0-9]', ' ', text)
    text_features += ngram(text_punc.split(), 1)
    return Counter(text_features)


به عنوان مثال سه خط کد زیر را اجرا میکنیم 

In [14]:
print(create_feature("I love you!"))
print(create_feature(" aly wins the gold!!!"))
print(create_feature(" aly wins the gold!!!!!", (1, 2)))

Counter({'i': 1, 'love': 1, 'you': 1, '!': 1})
Counter({'aly': 1, 'wins': 1, 'the': 1, 'gold': 1, '!!!': 1})
Counter({'aly': 1, 'wins': 1, 'the': 1, 'gold': 1, 'aly wins': 1, 'wins the': 1, 'the gold': 1, '!!!!!': 1})


لیبل ها در این پروژه به صورت وان هات اینکودینگ است این تابع برای تبدیل این لیبل به یک کلمه که همان اسم احساس است مورد استفاده قرار میگیرد

In [15]:
def convert_label(item, name): 
    items = list(map(float, item.split()))
    label = ""
    for idx in range(len(items)): 
        if items[idx] == 1: 
            label += name[idx] + " "
    
    return label.strip()


در این جا یک لیست ب اسم های احساس های موجود برای هر متن که در اخر همان لیبل ها میشود ایجاد میکنیم 

In [17]:
emotions = ["joy", 'fear', "anger", "sadness", "disgust", "shame", "guilt"]

حالا زمان ایجاد دیتای ویژگی ها و تارگت (لیبل ) ان ها است که از دو تابع از قبل نوشته شده استفاده میکنیم

In [18]:
X_all = []
y_all = []
for label, text in data:
    y_all.append(convert_label(label, emotions))
    X_all.append(create_feature(text, nrange=(1, 4)))

print("features example: ")
print(X_all[0])
print("Label example:")
print(y_all[0])

features example: 
Counter({'time': 2, 'we': 2, 'met': 2, 'during': 1, 'the': 1, 'period': 1, 'of': 1, 'falling': 1, 'in': 1, 'love': 1, 'each': 1, 'that': 1, 'and': 1, 'especially': 1, 'when': 1, 'had': 1, 'not': 1, 'for': 1, 'a': 1, 'long': 1, 'during the': 1, 'the period': 1, 'period of': 1, 'of falling': 1, 'falling in': 1, 'in love': 1, 'love each': 1, 'each time': 1, 'time that': 1, 'that we': 1, 'we met': 1, 'met and': 1, 'and especially': 1, 'especially when': 1, 'when we': 1, 'we had': 1, 'had not': 1, 'not met': 1, 'met for': 1, 'for a': 1, 'a long': 1, 'long time': 1, 'during the period': 1, 'the period of': 1, 'period of falling': 1, 'of falling in': 1, 'falling in love': 1, 'in love each': 1, 'love each time': 1, 'each time that': 1, 'time that we': 1, 'that we met': 1, 'we met and': 1, 'met and especially': 1, 'and especially when': 1, 'especially when we': 1, 'when we had': 1, 'we had not': 1, 'had not met': 1, 'not met for': 1, 'met for a': 1, 'for a long': 1, 'a long t

این کد برای جدا سازی دیتای مورد نیاز برای اموزش مدل و تست میباشد

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 123)

این تابع نوع کلاس طبقه بندی و دیتاهای مورد نیاز را میگیرد و درصد موفقیت روی دیتای اموزش و تست برمیگرداند

In [20]:
def train_test(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    return train_acc, test_acc


در این بخش چون دیتای ما بصورت دیکشنری است ابتدا توسط این کلاس به یک وکتور که ورودی مدل ماشین لرنینگ ما است اماده میشود

In [21]:
from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer(sparse = True)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


انواع کلاس های طبقه بندی ایجاد و لیستی از انها برای دادن بع تابع بالا ایجاد میشود

In [22]:
# Classifiers 
svc = SVC()
lsvc = LinearSVC(random_state=123)
rforest = RandomForestClassifier(random_state=123)
dtree = DecisionTreeClassifier()

clifs = [svc, lsvc, rforest, dtree]

و در این مرحله یک حلقه برای اعمال هر 4 کلاس طبقه بندی به تابع قبل ایجاد میکنیم و نتایج هرکدام رو در خروجی چاپ میکنیم 

In [ ]:
print("| {:25} | {} | {} |".format("Classifier", "Training Accuracy", "Test Accuracy"))
print("| {} | {} | {} |".format("-"*25, "-"*17, "-"*13))
for clf in clifs: 
    clf_name = clf.__class__.__name__
    train_acc, test_acc = train_test(clf, X_train, X_test, y_train, y_test)
    print("| {:25} | {:17.7f} | {:13.7f} |".format(clf_name, train_acc, test_acc))

l = ["joy", 'fear', "anger", "sadness", "disgust", "shame", "guilt"]
l.sort()
label_freq = {}
for label, _ in data: 
    label_freq[label] = label_freq.get(label, 0) + 1

و در اخرین بخش این پروژه پیش بینی شده توسط ماشین به یک ایموجی مشخص شده در زیر تبدیل شده و در رابط وب نمایش داده میشود 

In [ ]:
# print the labels and their counts in sorted order 
for l in sorted(label_freq, key=label_freq.get, reverse=True):
    print("{:10}({})  {}".format(convert_label(l, emotions), l, label_freq[l]))

emoji_dict = {"joy":"😂", "fear":"😱", "anger":"😠", "sadness":"😢", "disgust":"😒", "shame":"😳", "guilt":"😳"}

texts = [t1]
for text in texts: 
    features = create_feature(text, nrange=(1, 4))
    features = vectorizer.transform(features)
    prediction = clf.predict(features)[0]
    st.write(emoji_dict[prediction])